<a href="https://colab.research.google.com/github/Pramsi/Can-t_tame_that_Python_Semester_Project/blob/main/Can't_tame_that_Python_Semester_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
#for data manipulation
import numpy as np
import pandas as pd

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Other tooling
import os

#NLP
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
!pip install --upgrade emoji
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 4.7 MB/s eta 0:00:00


#Import Datafiles

In [6]:
# create kaggle directory and add kaggle.json
from google.colab import files
files.upload()

! mkdir -p ~/.kaggle #create kaggle directory
! mv kaggle.json ~/.kaggle/ #move json file there
! chmod 600 ~/.kaggle/kaggle.json #change permission of the file

Saving kaggle (1).json to kaggle (1).json
mv: cannot stat 'kaggle.json': No such file or directory


In [7]:
#create gitignore and add kaggle
%%writefile .gitignore
.kaggle/

Overwriting .gitignore


In [9]:
#upload brexit dataset
!kaggle datasets download -d visalakshiiyer/twitter-data-brexit
! mkdir brexitTweets #create folder with files
! unzip twitter-data-brexit.zip -d brexitTweets #unzip data and save in folder

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 70, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 1493, in dataset_download_cli
    self.dataset_download_files(dataset,
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 1447, in dataset_download_files
    self.download_file(response, outfile, quiet, not force)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 1834, in download_file
    size = int(response.headers['Content-Length'])
  File "/usr/local/lib/python3.10/dist-packages/urllib3/_collections.py", line 258, in __getitem__
    val = self._container[key.lower()]
KeyError: 'content-length'
mkdir: cannot create directory ‘brexitTweets’: File exists
unzip:  cannot find or open twitter-data-brexit

In [11]:
!ls -a


 .   ..   brexitTweets	 .config   .gitignore  'kaggle (1).json'   sample_data


In [10]:
#Load Datasets
dfAnti = pd.read_csv("brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv")
dfPro = pd.read_csv("brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv")

FileNotFoundError: ignored

In [ ]:
#Check Data
dfAnti.head()

#Preprocessing

In [ ]:
# label pro and anti tweets and concatenate the two files
dfAnti['Position'] = 0
dfPro['Position'] = 1

df = pd.concat([dfAnti, dfPro])

#drop unwanted data
df.drop(['Unnamed: 0', 'Date', 'Headline', 'URL', 'Opening Text', 'Source', 'Influencer', 'Country', 'Subregion', 'Language', 'Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo', 'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership', 'Engagement', 'AVE', 'Key Phrases', 'Input Name', 'Keywords', 'Twitter Authority', 'Tweet Id', 'Twitter Id', 'Twitter Client', 'Twitter Screen Name', 'User Profile Url', 'Twitter Bio', 'Twitter Followers', 'Twitter Following', 'Alternate Date Format', 'Time', 'State', 'City', 'Document Tags'], axis = 1, inplace=True)

#count the data
df.Position.value_counts()

In [ ]:
#Clean Hit Sentence - Remove Retweets
df_no_rt = df[df['Hit Sentence'].str.contains("RT @")==False]
df_no_qt = df_no_rt[df_no_rt['Hit Sentence'].str.contains("QT @")==False]

#how many tweets were retweeted and qoted?
print(f'Amount of retweets and quote tweets in percentage:{round(100*(1-len(df_no_qt)/len(df)),1)}%')
print(df_no_qt)

In [ ]:
#How many Pro and Anti Tweets are left
df_no_qt.Position.value_counts()

In [ ]:
#Preprocess Helper Functions

#to lower case
def lowercase(tweet: str) -> str:
  preprocessedTweet = str.lower(tweet)
  return preprocessedTweet

#remove hashtags
def remove_hashtags(tweet: str) -> str:
  preprocessedTweet = re.sub('#[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove mentions
def remove_mentions(tweet: str) -> str:
  preprocessedTweet = re.sub('@[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove emojis
def remove_emojis(tweet: str) -> str:
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub('', tweet)

#remove stopwords
def remove_stopwords(tweet: str) -> str:
    word_list = tweet.split()
    filtered_words = [word for word in word_list if word not in set(stopwords.words('english'))]
    return ' '.join(filtered_words)

# Function to combine all preprocessing steps
def preprocess_tweet(tweet: str) -> str:
    tweet = lowercase(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_mentions(tweet)
    tweet = remove_stopwords(tweet)
    tweet = remove_emojis(tweet)
    return tweet

In [ ]:
# no retweets and no quote tweets / no cleaned column yet
df_no_qt.head()

In [ ]:
# create cleaned tweet column
df_no_qt['tweet_clean'] = df_no_qt['Hit Sentence'].apply(preprocess_tweet)

In [ ]:
# with cleaned tweet column
df_no_qt.head()

#Split and Vectorize the data

In [ ]:
# Set X and y label
X_preprocessed = df_no_qt["tweet_clean"]
y = df_no_qt["Sentiment"]

In [ ]:
# Encode the y labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_encoded, test_size=0.3, random_state=42)

In [ ]:
#Vectorize the data
tfidf_vectorizer = TfidfVectorizer(stop_words='english') #make sure that all stopwords are excluded, stopword list might differ
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

count_vectorizer = CountVectorizer(stop_words="english") #make sure that all stopwords are excluded
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

#Train Models with Tfidf Vectorizer

In [ ]:
# Initialize a Support Vector Classifier (SVC) with a linear kernel
# svm_linear_tfidf = SVC(kernel='linear', C=1)

# Train the classifier using the tfidf training data and training labels
# svm_linear_tfidf.fit(tfidf_train, y_train)

In [ ]:
# Initialize a Logistic Regression Model
log_reg_tfidf = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)

#Train the Linear Regression Model using the tfidf data
log_reg_tfidf.fit(tfidf_train, y_train)

In [ ]:
# Initialize a Random Forest Classifier
random_forest_tfidf = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)

# Fit the Random Forest Classifier on the training data
random_forest_tfidf.fit(tfidf_train, y_train)

In [ ]:
# Initialize Naive Bayes Classifier
nb_classifier_tfidf = MultinomialNB()

# Fit the Naive Bayes Classifier with the tfidf data
nb_classifier_tfidf.fit(tfidf_train, y_train)

Make predictions with Tfidf vectorized data

In [ ]:
# Predict the labels for the tfidf vectorized data using a linear SVM model
# y_pred_svm_tfidf = svm_linear_tfidf.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a logistic regression model
y_pred_logreg_tfidf = log_reg_tfidf.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a Random Forest Classifier
y_pred_forest_tfidf = random_forest_tfidf.predict(tfidf_test)

# Predict the labels for the tfidf vectorized data using a Naive Bayes Classifier
y_pred_nb_tfidf = nb_classifier_tfidf.predict(tfidf_test)

Evaluate our Models with the Tfidf vectorized data

In [ ]:
#SVM Evaluation
# print(f"SVM Evaluation:")
# Calculate and print the accuracy score
# accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)
# print(f"Accuracy: {accuracy_svm_tfidf}")

#  Calculate and print the F1 score
# f1_svm_tfidf = f1_score(y_test, y_pred_svm_tfidf, average='macro')
# print(f"F1 Score: {f1_svm_tfidf}")

#  Calculate and print the precision score
# precision_svm_tfidf = precision_score(y_test, y_pred_svm_tfidf, average='macro')
# print(f"Precision: {precision_svm_tfidf}")

#  Calculate and print the recall score
# recall_svm_tfidf = recall_score(y_test, y_pred_svm_tfidf, average='macro')
# print(f"Recall: {recall_svm_tfidf}")

# print(f"---------------------------------------------")

print(f"Logistic Regression Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_logreg_tfidf = accuracy_score(y_test, y_pred_logreg_tfidf)
print(f"Accuracy: {accuracy_logreg_tfidf}")

# Calculate and print the F1 score
f1_logreg_tfidf = f1_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"F1 Score: {f1_logreg_tfidf}")

# Calculate and print the precision score
precision_logreg_tfidf = precision_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"Precision: {precision_logreg_tfidf}")

# Calculate and print the recall score
recall_logreg_tfidf = recall_score(y_test, y_pred_logreg_tfidf, average='macro')
print(f"Recall: {recall_logreg_tfidf}")

print(f"---------------------------------------------")

print(f"Random Forest Evaluation:")
#Random Forest Evaluation
# Calculate and print the accuracy score
accuracy_forest_tfidf = accuracy_score(y_test, y_pred_forest_tfidf)
print(f"Accuracy: {accuracy_forest_tfidf}")

# Calculate and print the F1 score
f1_forest_tfidf = f1_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"F1 Score: {f1_forest_tfidf}")

# Calculate and print the precision score
precision_forest_tfidf = precision_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"Precision: {precision_forest_tfidf}")

# Calculate and print the recall score
recall_forest_tfidf = recall_score(y_test, y_pred_forest_tfidf, average='macro')
print(f"Recall: {recall_forest_tfidf}")

print(f"---------------------------------------------")

print(f"Naive Bayes Classifier Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_nb_tfidf = accuracy_score(y_test, y_pred_nb_tfidf)
print(f"Accuracy: {accuracy_nb_tfidf}")

# Calculate and print the F1 score
f1_nb_tfidf = f1_score(y_test, y_pred_nb_tfidf, average='macro')
print(f"F1 Score: {f1_nb_tfidf}")

# Calculate and print the precision score
precision_nb_tfidf = precision_score(y_test, y_pred_nb_tfidf, average='macro', zero_division=1)
print(f"Precision: {precision_nb_tfidf}")

# Calculate and print the recall score
recall_nb_tfidf = recall_score(y_test, y_pred_nb_tfidf, average='macro', zero_division=1)
print(f"Recall: {recall_nb_tfidf}")

print(f"---------------------------------------------")

#Train Models with Count Vectorizer

In [ ]:
# Initialize a Support Vector Classifier (SVC) with a linear kernel
# svm_linear_count = SVC(kernel='linear', C=1)

# Train the classifier using the tfidf training data and training labels
# svm_linear_count.fit(count_train, y_train)

In [ ]:
# Initialize a Logistic Regression Model
log_reg_count = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)

#Train the Linear Regression Model using the tfidf data
log_reg_count.fit(count_train, y_train)

In [ ]:
# Initialize a Random Forest Classifier
random_forest_count = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)

# Fit the Random Forest Classifier on the training data
random_forest_count.fit(count_train, y_train)

In [ ]:
# Initialize Naive Bayes Classifier
nb_classifier_count = MultinomialNB()

# Fit the Naive Bayes Classifier with the tfidf data
nb_classifier_count.fit(count_train, y_train)

Make predictions with Count vectorized data

In [ ]:
# Predict the labels for the count vectorized data using a linear SVM model
# y_pred_svm_count = svm_linear_count.predict(count_test)

# Predict the labels for the count vectorized data using a logistic regression model
y_pred_logreg_count = log_reg_count.predict(count_test)

# Predict the labels for the count vectorized data using a Random Forest Classifier
y_pred_forest_count = random_forest_count.predict(count_test)

# Predict the labels for the count vectorized data using a Naive Bayes Classifier
y_pred_nb_count = nb_classifier_count.predict(count_test)

Evaluate our Models with the Count vectorized data

In [ ]:
#SVM Evaluation
# print(f"SVM Evaluation:")
# Calculate and print the accuracy score
# accuracy_svm_count = accuracy_score(y_test, y_pred_svm_count)
# print(f"Accuracy: {accuracy_svm_count}")

# Calculate and print the F1 score
# f1_svm_count = f1_score(y_test, y_pred_svm_count, average='macro')
# print(f"F1 Score: {f1_svm_count}")

# Calculate and print the precision score
# precision_svm_count = precision_score(y_test, y_pred_svm_count, average='macro')
# print(f"Precision: {precision_svm_count}")

# Calculate and print the recall score
# recall_svm_count = recall_score(y_test, y_pred_svm_count, average='macro')
# print(f"Recall: {recall_svm_count}")

# print(f"---------------------------------------------")

print(f"Logistic Regression Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_logreg_count = accuracy_score(y_test, y_pred_logreg_count)
print(f"Accuracy: {accuracy_logreg_count}")

# Calculate and print the F1 score
f1_logreg_count = f1_score(y_test, y_pred_logreg_count, average='macro')
print(f"F1 Score: {f1_logreg_count}")

# Calculate and print the precision score
precision_logreg_count = precision_score(y_test, y_pred_logreg_count, average='macro')
print(f"Precision: {precision_logreg_count}")

# Calculate and print the recall score
recall_logreg_count = recall_score(y_test, y_pred_logreg_count, average='macro')
print(f"Recall: {recall_logreg_count}")

print(f"---------------------------------------------")

print(f"Random Forest Evaluation:")
#Random Forest Evaluation
# Calculate and print the accuracy score
accuracy_forest_count = accuracy_score(y_test, y_pred_forest_count)
print(f"Accuracy: {accuracy_forest_count}")

# Calculate and print the F1 score
f1_forest_count = f1_score(y_test, y_pred_forest_count, average='macro')
print(f"F1 Score: {f1_forest_count}")

# Calculate and print the precision score
precision_forest_count = precision_score(y_test, y_pred_forest_count, average='macro')
print(f"Precision: {precision_forest_count}")

# Calculate and print the recall score
recall_forest_count = recall_score(y_test, y_pred_forest_count, average='macro')
print(f"Recall: {recall_forest_count}")

print(f"---------------------------------------------")

print(f"Naive Bayes Classifier Evaluation:")
#Logistic Regression Evaluation
# Calculate and print the accuracy score
accuracy_nb_count = accuracy_score(y_test, y_pred_nb_count)
print(f"Accuracy: {accuracy_nb_count}")

# Calculate and print the F1 score
f1_nb_count = f1_score(y_test, y_pred_nb_count, average='macro')
print(f"F1 Score: {f1_nb_count}")

# Calculate and print the precision score
precision_nb_count = precision_score(y_test, y_pred_nb_count, average='macro', zero_division=1)
print(f"Precision: {precision_nb_count}")

# Calculate and print the recall score
recall_nb_count = recall_score(y_test, y_pred_nb_count, average='macro', zero_division=1)
print(f"Recall: {recall_nb_count}")

print(f"---------------------------------------------")

In [ ]:
parameters_log = {'C': [0.1, 1, 10, 100]}

# Initialize a Logistic Regression Classifier
log_reg_count = LogisticRegression(C=1, solver='lbfgs', max_iter=1000)

# Set up GridSearchCV with the SVC, parameter grid, 5-fold cross-validation, and F1 scoring
grid_log_reg_count = GridSearchCV(log_reg_count, parameters_log, cv=5, scoring="f1")

# Train the GridSearchCV on the scaled training data
grid_log_reg_count.fit(count_train, y_train)

In [ ]:
print(f"Best Parameters: {grid_log_reg_count.best_params_}")
print(f"Best F1 - Score: {grid_log_reg_count.best_score_}")